In [2]:
import pandas as pd
import json
from datetime import datetime

df1 = pd.read_csv('bd.csv', sep='\t',
                 names=["id", "datetime", "lat", "long"])
# df2 = pd.read_csv('542', sep='\t',
#                  names=["gid", "datetime", "lat", "long"])

In [3]:
df2 = pd.read_csv('549', sep='\t',
                 names=["gid", "datetime", "lat", "long"])

In [4]:
print(df1)
print('----------inter-data-frame\n')
df2

           id             datetime       lat       long
0           1  2015-03-04 00:35:16  4.870147  45.772140
1           1  2015-03-04 00:35:48  4.870218  45.772095
2           1  2015-03-04 00:35:49  4.870210  45.772072
3           1  2015-03-04 00:35:50  4.870210  45.772072
4           1  2015-03-04 00:35:52  4.870210  45.772072
...       ...                  ...       ...        ...
34551844  110  2015-03-12 16:23:21  2.343094  48.891650
34551845  110  2015-03-12 16:23:22  2.343094  48.891650
34551846  110  2015-03-12 16:23:24  2.343094  48.891649
34551847  110  2015-03-12 16:23:25  2.343094  48.891649
34551848  110  2015-03-12 19:29:04  2.343127  48.891772

[34551849 rows x 4 columns]
----------inter-data-frame



,gid,datetime,lat,long
0,"]'`(#^=#*""@]$}'",2015-03-19 11:59:59,4.865606,45.782156
1,"""\/*),>$.%\}!~~",2015-05-06 11:59:59,4.865255,45.782215
2,"[_$$,];<|;$$.@`",2015-05-07 20:59:59,5.279566,45.585595
3,">=,<&]-&/""_~#'|",2015-03-12 20:59:59,4.865536,45.755755
4,")'=*+.):+!&'""[-",2015-04-15 23:23:52,4.869266,45.775416
...,...,...,...,...
34551844,"(?),_(|``**!{\\",2015-03-21 11:59:59,4.282485,45.565725
34551845,"&_""+>_}|&*@!}@'",2015-04-17 23:40:06,4.282516,45.565686
34551846,.[_;~*;&\&`(><`,2015-03-21 11:59:59,5.053295,45.818785
34551847,")"":+:+<+__!:#]'",2015-04-08 14:59:59,4.087676,46.018985


In [5]:
df2=df2[df2['gid']!='DEL']

df2['week']=pd.to_datetime(df2['datetime']).dt.isocalendar().week

df1['week']=pd.to_datetime(df1['datetime']).dt.isocalendar().week

<ipython-input-5-e18dad3bf982>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['week']=pd.to_datetime(df2['datetime']).dt.isocalendar().week


In [6]:
df2.dtypes

gid          object
datetime     object
lat         float64
long        float64
week         UInt32
dtype: object

In [7]:
# import numpy as np
# df2.shape
# df2[df2[['long']].applymap(np.isreal).all(1)]
# df2["long"] = pd.to_numeric(df2["long"])

df1.dtypes

id            int64
datetime     object
lat         float64
long        float64
week         UInt32
dtype: object

In [8]:
df1['datetime'] = df1['datetime'].str.slice(stop=16)
df1['datetime'] = pd.to_datetime(df1['datetime'], format='%Y-%m-%d %H:%M')

In [9]:
df2['datetime'] = df2['datetime'].str.slice(stop=16)
df2['datetime'] = pd.to_datetime(df2['datetime'], format='%Y-%m-%d %H:%M')

<ipython-input-9-978e1e8be738>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['datetime'] = df2['datetime'].str.slice(stop=16)
<ipython-input-9-978e1e8be738>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['datetime'] = pd.to_datetime(df2['datetime'], format='%Y-%m-%d %H:%M')


In [10]:
import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [33]:
weeks=df1['week'].unique()
big={}
for w in weeks:
    print(f'week {w}')
    j=0
    dftmp1=df1[df1['week']==w]
    dftmp2=df2[df2['week']==w]
    
    dftmp1["week_day"] = dftmp1["datetime"].dt.isocalendar().day    
    dftmp2["week_day"] = dftmp2["datetime"].dt.isocalendar().day    
    
    js1= dftmp1[dftmp1["week_day"] <= 5]                                 # jour de travail initial
    wk1= dftmp1.drop(dftmp1[dftmp1["week_day"] <= 5].index)                # wk initial
    
    js2= dftmp2[dftmp2["week_day"] <= 5]                                 # jour de travail anonymise
    wk2= dftmp2.drop(dftmp2[dftmp2["week_day"] <= 5].index)                # wk anonymise
    
    js1_size=js1.groupby('id', as_index=False).size()
    wk1_size=wk1.groupby('id', as_index=False).size()
    
    js2_size=js2.groupby('gid', as_index=False).size()
    wk2_size=wk2.groupby('gid', as_index=False).size()
    
    wk1_size = pd.merge(js1_size, wk1_size, on = ['id'],how = 'left').sort_values(['size_x','size_y'], ascending=[True, True])
    wk1_size = wk1_size.fillna(0)
    
    wk2_size = pd.merge(js2_size, wk2_size, on = ['gid'],how = 'left').sort_values(['size_x','size_y'], ascending=[True, True])
    wk2_size = wk2_size.fillna(0)
    
    name="2015-"+str(w)
    
    
    for i in wk2_size['gid'].to_numpy():
        origId=str(wk1_size.iloc[j,0])
        pred=wk2_size.iloc[j,0]
        if origId in big:
            big[origId][name]=[str(pred)]
        else:
            dictmp={}
            dictmp[name]=[str(pred)]
            tmp_dict={
                origId:dictmp
            }
        big.update(tmp_dict)
        j+=1

week 10


<ipython-input-33-3e8791a9846c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp1["week_day"] = dftmp1["datetime"].dt.isocalendar().day
<ipython-input-33-3e8791a9846c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp2["week_day"] = dftmp2["datetime"].dt.isocalendar().day


week 11


<ipython-input-33-3e8791a9846c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp1["week_day"] = dftmp1["datetime"].dt.isocalendar().day
<ipython-input-33-3e8791a9846c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp2["week_day"] = dftmp2["datetime"].dt.isocalendar().day


week 12


...

<ipython-input-33-3e8791a9846c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp1["week_day"] = dftmp1["datetime"].dt.isocalendar().day
<ipython-input-33-3e8791a9846c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp2["week_day"] = dftmp2["datetime"].dt.isocalendar().day


week 19


<ipython-input-33-3e8791a9846c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp1["week_day"] = dftmp1["datetime"].dt.isocalendar().day
<ipython-input-33-3e8791a9846c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp2["week_day"] = dftmp2["datetime"].dt.isocalendar().day


week 20


<ipython-input-33-3e8791a9846c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp1["week_day"] = dftmp1["datetime"].dt.isocalendar().day
<ipython-input-33-3e8791a9846c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftmp2["week_day"] = dftmp2["datetime"].dt.isocalendar().day


In [32]:
with open('549.json', 'w') as f:
    json.dump(big, f)

In [208]:
# df_grouped1=df_grouped1.sort_values('size')
# df_grouped2=df_grouped2.sort_values('size')
# df2.groupby('id').mean()
# i==1 et w==10
# dftmp1=df1[df['week']==10]
# latmp1=dftmp1[dftmp1['id']==1]['lat'].mean()
# dftmp2=df2[df['week']==10]
# mean_lat_week_ano=dftmp2.groupby('gid', as_index=False)['lat'].mean()
# nearest_value=find_nearest(mean_lat_week_ano["lat"].to_numpy(),latmp1 )
# nearest_value
# pred=mean_lat_week_ano[(mean_lat_week_ano['lat']>=nearest_value-0.001) & (mean_lat_week_ano['lat']<=nearest_value+0.001)]
# pred
# dftmp2.head(10)
# df2.groupby('gid', as_index=False)['lat'].mean()
# dftest1=df1[df1['id']==1]
# dftest1['lat'].mean()

In [209]:
# str(pred)
# df_grouped1
# df_grouped2.iloc[j,0:1].gid

In [210]:
# big={}
# j=0
# for i in df_grouped1['id'].unique():
#     ret= getPred(i)
#     tmp_dict={
#         str(i):
#     }
#     j+=1
#     big.update(tmp_dict)

In [211]:
# big